# Figure 1C - YARN Normalization Version

A `heatplot` representing similarity in the fold-changes between male and female samples, with the values in the heatmap being the correlation between the vectors of fold changes of the tissues. </b>

We downloaded the GTEx version 8.0 RNA-seq and genotype data (phs000424.v8.v2), released 2019-08-26.
We used YARN (https://bioconductor.org/packages/release/bioc/html/yarn.html), uploading the downloadGTEx function
to download this release, and used it to perform quality control, gene filtering and normalization pre-processing on the
GTEx RNA-seq data, as described in (Paulson et al, 2017).   This pipelines tested for sample sex-misidentification, 
merged related sub-tissues, performed tissue-aware normalization using qsmooth (Hicks et al, 2017).

## Loading dependencies

In [1]:
library(downloader)
library(readr)
library(edgeR)
library(biomaRt)
library(DBI) # v >= 1.1.0 required for biomaRt
library(devtools)
library(yarn)
#install.packages("statmod")
library(statmod)
Sys.setenv(TAR = "/bin/tar") # for gzfile

Loading required package: limma
Warning message:
“package ‘DBI’ was built under R version 3.6.2”Loading required package: Biobase
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following object is masked from ‘package:limma’:

    plotMA

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdi

Begin here if you have already run this and created the `data/gtex.rds` file

Please `git clone` the repository and start working after changing to this as your working directory (`cd lifebitCloudOSDRE`). The `data` subdirectory, along with all other paths used in this Notebook are relative to the `lifebitCloudOSDRE` repository.

In [2]:
#NOTE TO USERS:
#Either run 1 or 2
#
#1. CAUTION! It requires some minutes to complete, also memory and enough storage
#obj <- yarn::downloadGTExV8(type='genes',file='../data/gtex.rds')
#saveRDS(obj, file = "../data/gtex.rds")
#
#or
#2. Retrieve the released location the already generated gtex.rds
# Run the following line in the data directory prior to the next line if you wish to use the already
# generated file.
# wget https://github.com/lifebit-ai/lifebitCloudOSDREgtex/releases/download/gtex_archive/gtex.rds
# and then run this line.
obj <- readRDS("../data/gtex.rds")

In [4]:
# confirm that it is an expression set.

In [5]:
class(obj)

[1] "ExpressionSet"
attr(,"package")
[1] "Biobase"

In [6]:
dim(phenoData(obj))

rowNames columnNames 
      17384          67

In [7]:
# for later - not sure attributes are actually successfully being added
#ensembl = useMart("ensembl",dataset="hsapiens_gene_ensembl")
#attributes = listAttributes(ensembl)

In [8]:
dim(obj)

Features  Samples 
   55878    17382

In [9]:
sample_names=as.vector(as.character(colnames(exprs(obj))))
head(sample_names)
length(sample_names)

[1] "GTEX-1117F-0226-SM-5GZZ7" "GTEX-1117F-0426-SM-5EGHI"
[3] "GTEX-1117F-0526-SM-5EGHJ" "GTEX-1117F-0626-SM-5N9CS"
[5] "GTEX-1117F-0726-SM-5GIEN" "GTEX-1117F-1326-SM-5EGHH"

[1] 17382

In [10]:
pheno_sample_names=as.vector(as.character(rownames(pData(obj))))
head(pheno_sample_names)
length(pheno_sample_names)

[1] "GTEX-1117F-0226-SM-5GZZ7" "GTEX-1117F-0426-SM-5EGHI"
[3] "GTEX-1117F-0526-SM-5EGHJ" "GTEX-1117F-0626-SM-5N9CS"
[5] "GTEX-1117F-0726-SM-5GIEN" "GTEX-1117F-1326-SM-5EGHH"

[1] 17384

Okay - for some reason our phenotype data is larger than our expression data - I've written Joe Paulson about that.
In the meantime, make sure that the two sets are aligned.

In [11]:
logical_match_names=pheno_sample_names %in% sample_names
length(logical_match_names)

[1] 17384

In [12]:
table(logical_match_names)


logical_match_names
FALSE  TRUE 
    2 17382 

In [13]:
pData(obj) <- (pData(obj)[logical_match_names==TRUE,])

In [14]:
dim(pData(obj))
dim(obj)

[1] 17382    67

Features  Samples 
   55878    17382

Now we want to replace all *dashes* with _underscores_

In [15]:
newSampID <- gsub('-','\\.',pData(obj)$SAMPID)

In [16]:
head (newSampID)

[1] "GTEX.1117F.0226.SM.5GZZ7" "GTEX.1117F.0426.SM.5EGHI"
[3] "GTEX.1117F.0526.SM.5EGHJ" "GTEX.1117F.0626.SM.5N9CS"
[5] "GTEX.1117F.0726.SM.5GIEN" "GTEX.1117F.1326.SM.5EGHH"

In [17]:
pData(obj)$SAMPID <- newSampID

In [18]:
colnames(pData(obj))

[1] "SAMPID"    "SMATSSCR"  "SMCENTER"  "SMPTHNTS"  "SMRIN"     "SMTS"     
 [7] "SMTSD"     "SMUBRID"   "SMTSISCH"  "SMTSPAX"   "SMNABTCH"  "SMNABTCHT"
[13] "SMNABTCHD" "SMGEBTCH"  "SMGEBTCHD" "SMGEBTCHT" "SMAFRZE"   "SMGTC"    
[19] "SME2MPRT"  "SMCHMPRS"  "SMNTRART"  "SMNUMGPS"  "SMMAPRT"   "SMEXNCRT" 
[25] "SM550NRM"  "SMGNSDTC"  "SMUNMPRT"  "SM350NRM"  "SMRDLGTH"  "SMMNCPB"  
[31] "SME1MMRT"  "SMSFLGTH"  "SMESTLBS"  "SMMPPD"    "SMNTERRT"  "SMRRNANM" 
[37] "SMRDTTL"   "SMVQCFL"   "SMMNCV"    "SMTRSCPT"  "SMMPPDPR"  "SMCGLGTH" 
[43] "SMGAPPCT"  "SMUNPDRD"  "SMNTRNRT"  "SMMPUNRT"  "SMEXPEFF"  "SMMPPDUN" 
[49] "SME2MMRT"  "SME2ANTI"  "SMALTALG"  "SME2SNSE"  "SMMFLGTH"  "SME1ANTI" 
[55] "SMSPLTRD"  "SMBSMMRT"  "SME1SNSE"  "SME1PCTS"  "SMRRNART"  "SME1MPRT" 
[61] "SMNUM5CD"  "SMDPMPRT"  "SME2PCTS"  "SUBJID"    "SEX"       "AGE"      
[67] "DTHHRDY"

In [19]:
head(exprs(obj))

,GTEX-1117F-0226-SM-5GZZ7,GTEX-1117F-0426-SM-5EGHI,GTEX-1117F-0526-SM-5EGHJ,GTEX-1117F-0626-SM-5N9CS,GTEX-1117F-0726-SM-5GIEN,GTEX-1117F-1326-SM-5EGHH,GTEX-1117F-2426-SM-5EGGH,GTEX-1117F-2526-SM-5GZY6,GTEX-1117F-2826-SM-5GZXL,GTEX-1117F-2926-SM-5GZYI,⋯,GTEX-ZZPU-1126-SM-5N9CW,GTEX-ZZPU-1226-SM-5N9CK,GTEX-ZZPU-1326-SM-5GZWS,GTEX-ZZPU-1426-SM-5GZZ6,GTEX-ZZPU-1826-SM-5E43L,GTEX-ZZPU-2126-SM-5EGIU,GTEX-ZZPU-2226-SM-5EGIV,GTEX-ZZPU-2426-SM-5E44I,GTEX-ZZPU-2626-SM-5E45Y,GTEX-ZZPU-2726-SM-5NQ8O
ENSG00000223972.5,0,0,0,0,0,0,0,0,0,0,⋯,0,2,0,0,0,0,0,0,1,1
ENSG00000227232.5,187,109,143,251,113,139,199,473,286,306,⋯,72,96,136,79,89,86,49,84,34,66
ENSG00000278267.1,0,0,1,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
ENSG00000243485.5,1,0,0,1,0,0,0,0,0,1,⋯,0,0,1,0,2,2,0,1,0,0
ENSG00000237613.2,0,0,0,0,0,0,0,1,0,0,⋯,1,0,0,0,0,0,0,0,0,0
ENSG00000268020.3,0,1,0,0,0,1,1,0,1,2,⋯,1,0,0,1,2,0,0,0,1,1


Now let us do the differential analysis - using EdgeR

In [ ]:
x <- exprs(obj)

In [ ]:
dim(x)

To use the DGEList function from EdgeR, we need to transpose our x so that the length of group is equal
to the number of columns in our counts (x).

You will get an error in DGEList (counts = x, group = group) if the length of group is not equal to the number of columns in counts

In [ ]:
group <- factor(pData(obj)$SEX)

In [ ]:
y <- DGEList(counts=x, group=group)

I keep running out of memory on this step - so on my laptop after calculating the DGEList
I saved it and now I uploaded it to this larger memory machine

In [ ]:
attributes(y)

In [ ]:
#caution this step takes a lot of memory and time

In [ ]:
y <- calcNormFactors(y)

In [ ]:
saveRDS(y, file = "../data/DGENormFactorsy.rds")

In [ ]:
# For Guy -- does this do what you are expecting -- I am confused because what you get when you
#        ask for the min (table(groups)) is the smaller sized group -- which in this case is 
#        female -- it will help the reader to know what you are doing here with the statement.
#        one can read what it is doing but not understand your objective.

In [ ]:
groups <- pData(obj)$SEX
keep.events <- rep(TRUE, nrow(y))
for (group in c(1,2)) {
    keep.events <- keep.events & 
                   rowSums(cpm(y[,groups %in% group]) > 1) >= 0.25*min(table(groups))
}


In [ ]:
reduced_y<- y[keep.events,]

In [ ]:
reduced_obj <- obj[keep.events==TRUE,]

In [ ]:
dim(reduced_obj)

In [ ]:
#saveRDS(reduced_y, file = "../data/reduced_y.rds")
#saveRDS(reduced_obj, file = "../data/reduced_obj.rds")
reduced_obj <- readRDS(file = "../data/reduced_obj.rds")
reduced_y   <- readRDS(file = "../data/reduced_y.rds")


In [ ]:
# separate the analysis by male and by female
# then do the differential analysis regarding tissue
# then do the differential analysis male tissue vs female tissue

In [ ]:
reduced_male   <- pData(reduced_obj)$SEX==1
reduced_female <- pData(reduced_obj)$SEX==2

In [ ]:
reduced_obj_male   <- reduced_obj[,reduced_male==TRUE]
reduced_obj_female <- reduced_obj[,reduced_female==TRUE]

In [ ]:
dim(reduced_obj_male)
dim(reduced_obj_female)

In [ ]:
tissue_groups_male <- factor(pData(reduced_obj_male)$SMTS)
tissue_groups_female <- factor(pData(reduced_obj_female)$SMTS)

In [ ]:
# good sanity check, the male set does not have any vaginas or uterus
table (tissue_groups_male)

In [ ]:
# and the females have no prostate or testis
table(tissue_groups_female)

In [ ]:
# Reproducing Guys results using the yarn expression object
# loop through the tissues and for those tissues that are shared between the two sexes
# perform a differential gene analysis on a per tissue basis


In [ ]:
tissue_groups <- factor(pData(reduced_obj)$SMTS)

In [ ]:
tissue_male_female <- tissue_groups_male %in% tissue_groups_female
table(tissue_male_female)

In [ ]:
tissue_shared_male_female <- factor(tissue_groups_male[tissue_male_female])
table(tissue_shared_male_female)

In [ ]:
# SEX is coded 1 == Male
#              2 == Female
sex = factor(pData(reduced_obj)$SEX)

In [ ]:
#tissue_shared_male_female
for (tissue in tissue_shared_male_female) {
# missing: Bladder
#    tissue        <- 'Bladder'
    tissue_true   <- pData(reduced_obj)$SMTS==tissue
    tissue_obj    <- reduced_obj[,tissue_true==TRUE]
    tissue_sex    <- factor(pData(tissue_obj)$SEX)
    tissue_design <- model.matrix(~tissue_sex)
    y_tissue      <- DGEList(counts=exprs(tissue_obj), group=tissue_sex)
    tissue_sex_array <- lapply(pData(reduced_obj)$SMTS, factor())
    y_tissue_array <- lapply(pData(reduced_obj)$SMTS,DGEList(counts=exprs(tissue_obj), group=tissue_sex)
    y_tissue      <- calcNormFactors(y_tissue)
    y_tissue_voom <- voom (y_tissue, tissue_design)
    fit_tissue    <- lmFit(y_tissue_voom, tissue_design)
    fit_tissue    <- eBayes(fit_tissue, robust=TRUE)
    results_tissue<- topTable(fit_tissue, coef='tissue_sex2', number=nrow(y_tissue))
    assign(paste("results",tissue, sep="_"),results_tissue)
    filename = paste(paste("../data", tissue,sep="/"),"DGE.txt",sep="_")
    write.table(results_tissue,filename,sep='\t',quote = F)
}

In [ ]:
filenames <- list.files("../data", pattern="*_DGE.txt", all.files=FALSE,
    full.names=FALSE)

In [ ]:
adipose_tissue_logFC_mat <- read.delim2("../data/Adipose\ Tissue_DGE.txt",stringsAsFactors = FALSE)
adrenal_gland_logFC_mat  <- read.delim2("../data/Adrenal\ Gland_DGE.txt",stringsAsFactors = FALSE)
bladder_logFC_mat        <- read.delim2("../data/Bladder_DGE.txt",stringsAsFactors = FALSE)
blood_logFC_mat          <- read.delim2("../data/Blood_DGE.txt",stringsAsFactors = FALSE)
blood_vessel_logFC_mat   <- read.delim2("../data/Blood\ Vessel_DGE.txt",stringsAsFactors = FALSE) 
brain_logFC_mat          <- read.delim2("../data/Brain_DGE.txt",stringsAsFactors = FALSE)
breast_logFC_mat         <- read.delim2("../data/Breast_DGE.txt",stringsAsFactors = FALSE)
colon_logFC_mat          <- read.delim2("../data/Colon_DGE.txt",stringsAsFactors = FALSE)
esophagus_logFC_mat      <- read.delim2("../data/Esophagus_DGE.txt",stringsAsFactors = FALSE)
heart_logFC_mat          <- read.delim2("../data/Heart_DGE.txt",stringsAsFactors = FALSE)
kidney_logFC_mat         <- read.delim2("../data/Kidney_DGE.txt",stringsAsFactors = FALSE)
liver_logFC_mat          <- read.delim2("../data/Liver_DGE.txt",stringsAsFactors = FALSE)
lung_logFC_mat           <- read.delim2("../data/Lung_DGE.txt",stringsAsFactors = FALSE)
muscle_logFC_mat         <- read.delim2("../data/Muscle_DGE.txt",stringsAsFactors = FALSE)
nerve_logFC_mat          <- read.delim2("../data/Nerve_DGE.txt",stringsAsFactors = FALSE)
pancreas_logFC_mat       <- read.delim2("../data/Pancreas_DGE.txt",stringsAsFactors = FALSE)
pituitary_logFC_mat      <- read.delim2("../data/Pituitary_DGE.txt",stringsAsFactors = FALSE)
salivary_gland_logFC_mat <- read.delim2("../data/Salivary\ Gland_DGE.txt",stringsAsFactors = FALSE)
skin_logFC_mat           <- read.delim2("../data/Skin_DGE.txt",stringsAsFactors = FALSE)
small_intestine_logFC_mat<- read.delim2("../data/Small\ Intestine_DGE.txt",stringsAsFactors = FALSE)
spleen_logFC_mat         <- read.delim2("../data/Spleen_DGE.txt",stringsAsFactors = FALSE)
stomach_logFC_mat        <- read.delim2("../data/Stomach_DGE.txt",stringsAsFactors = FALSE)
thyroid_logFC_mat        <- read.delim2("../data/Thyroid_DGE.txt",stringsAsFactors = FALSE)

In [ ]:
adipose_tissue_logFC <- as.matrix(as.numeric(adipose_tissue_logFC_mat$logFC),nrow=16794,ncol=1)

m <- adipose_tissue_logFC[order(rownames(adipose_tissue_logFC_mat)),]

In [ ]:
adipose_tissue_logFC_mat       <- adipose_tissue_logFC_mat[order(rownames(adipose_tissue_logFC_mat)),]
adipose_tissue_logFC           <- as.matrix(as.numeric(adipose_tissue_logFC_mat$logFC),nrow=16794,ncol=1)
rownames(adipose_tissue_logFC) <- rownames(adipose_tissue_logFC_mat)

adrenal_gland_logFC_mat       <- adrenal_gland_logFC_mat[order(rownames(adrenal_gland_logFC_mat)),]
adrenal_gland_logFC           <- as.matrix(as.numeric(adrenal_gland_logFC_mat$logFC),nrow=16794,ncol=1)
rownames(adrenal_gland_logFC) <- rownames(adrenal_gland_logFC_mat)

bladder_logFC_mat       <- bladder_logFC_mat[order(rownames(bladder_logFC_mat)),]
bladder_logFC           <- as.matrix(as.numeric(bladder_logFC_mat$logFC),nrow=16794,ncol=1)
rownames(bladder_logFC) <- rownames(bladder_logFC_mat)

blood_logFC_mat       <- blood_logFC_mat[order(rownames(blood_logFC_mat)),]
blood_logFC           <- as.matrix(as.numeric(blood_logFC_mat$logFC),nrow=16794,ncol=1)
rownames(blood_logFC) <- rownames(blood_logFC_mat)

blood_vessel_logFC_mat       <- blood_vessel_logFC_mat[order(rownames(blood_vessel_logFC_mat)),]
blood_vessel_logFC           <- as.matrix(as.numeric(blood_vessel_logFC_mat$logFC),nrow=16794,ncol=1)
rownames(blood_vessel_logFC) <- rownames(blood_vessel_logFC_mat)

brain_logFC_mat      <- brain_logFC_mat[order(rownames(brain_logFC_mat)),]
brain_logFC          <- as.matrix(as.numeric(brain_logFC_mat$logFC),nrow=16794,ncol=1)
rownames(brain_logFC)<- rownames(brain_logFC_mat)

breast_logFC_mat       <- breast_logFC_mat[order(rownames(breast_logFC_mat)),]
breast_logFC           <- as.matrix(as.numeric(breast_logFC_mat$logFC),nrow=16794,ncol=1)
rownames(breast_logFC) <- rownames(breast_logFC_mat)

colon_logFC_mat       <- colon_logFC_mat[order(rownames(colon_logFC_mat)),]
colon_logFC           <- as.matrix(as.numeric(colon_logFC_mat$logFC),nrow=16794,ncol=1)
rownames(colon_logFC) <- rownames(colon_logFC_mat)

esophagus_logFC_mat       <- esophagus_logFC_mat[order(rownames(esophagus_logFC_mat)),]
esophagus_logFC           <- as.matrix(as.numeric(esophagus_logFC_mat$logFC),nrow=16794,ncol=1)
rownames(esophagus_logFC) <- rownames(esophagus_logFC_mat)

heart_logFC_mat       <- heart_logFC_mat[order(rownames(heart_logFC_mat)),]
heart_logFC           <- as.matrix(as.numeric(heart_logFC_mat$logFC),nrow=16794,ncol=1)
rownames(heart_logFC) <- rownames(heart_logFC_mat)

kidney_logFC_mat       <- kidney_logFC_mat[order(rownames(kidney_logFC_mat)),]
kidney_logFC           <- as.matrix(as.numeric(kidney_logFC_mat$logFC),nrow=16794,ncol=1)
rownames(kidney_logFC) <- rownames(kidney_logFC_mat)

liver_logFC_mat       <- liver_logFC_mat[order(rownames(liver_logFC_mat)),]
liver_logFC           <- as.matrix(as.numeric(liver_logFC_mat$logFC),nrow=16794,ncol=1)
rownames(liver_logFC) <- rownames(liver_logFC_mat)

lung_logFC_mat       <- lung_logFC_mat[order(rownames(lung_logFC_mat)),]
lung_logFC           <- as.matrix(as.numeric(lung_logFC_mat$logFC),nrow=16794,ncol=1)
rownames(lung_logFC) <- rownames(lung_logFC_mat)

muscle_logFC_mat      <- muscle_logFC_mat[order(rownames(muscle_logFC_mat)),]
muscle_logFC          <- as.matrix(as.numeric(muscle_logFC_mat$logFC),nrow=16794,ncol=1)
rownames(muscle_logFC)<- rownames(muscle_logFC_mat)

nerve_logFC_mat       <- nerve_logFC_mat[order(rownames(nerve_logFC_mat)),]
nerve_logFC           <- as.matrix(as.numeric(nerve_logFC_mat$logFC),nrow=16794,ncol=1)
rownames(nerve_logFC) <- rownames(nerve_logFC_mat)

pancreas_logFC_mat       <- pancreas_logFC_mat[order(rownames(pancreas_logFC_mat)),]
pancreas_logFC           <- as.matrix(as.numeric(pancreas_logFC_mat$logFC),nrow=16794,ncol=1)
rownames(pancreas_logFC) <- rownames(pancreas_logFC_mat)

pituitary_logFC_mat       <- pituitary_logFC_mat[order(rownames(pituitary_logFC_mat)),]
pituitary_logFC           <- as.matrix(as.numeric(pituitary_logFC_mat$logFC),nrow=16794,ncol=1)
rownames(pituitary_logFC) <- rownames(pituitary_logFC_mat)

salivary_gland_logFC_mat       <- salivary_gland_logFC_mat[order(rownames(salivary_gland_logFC_mat)),]
salivary_gland_logFC           <- as.matrix(as.numeric(salivary_gland_logFC_mat$logFC),nrow=16794,ncol=1)
rownames(salivary_gland_logFC) <- rownames(salivary_gland_logFC_mat)

skin_logFC_mat       <- skin_logFC_mat[order(rownames(skin_logFC_mat)),]
skin_logFC           <- as.matrix(as.numeric(skin_logFC_mat$logFC),nrow=16794,ncol=1)
rownames(skin_logFC) <- rownames(skin_logFC_mat)

small_intestine_logFC_mat       <- small_intestine_logFC_mat[order(rownames(small_intestine_logFC_mat)),]
small_intestine_logFC           <- as.matrix(as.numeric(small_intestine_logFC_mat$logFC),nrow=16794,ncol=1)
rownames(small_intestine_logFC) <- rownames(small_intestine_logFC_mat)

spleen_logFC_mat       <- spleen_logFC_mat[order(rownames(spleen_logFC_mat)),]
spleen_logFC           <- as.matrix(as.numeric(spleen_logFC_mat$logFC),nrow=16794,ncol=1)
rownames(spleen_logFC) <- rownames(spleen_logFC_mat)

stomach_logFC_mat       <- stomach_logFC_mat[order(rownames(stomach_logFC_mat)),]
stomach_logFC           <- as.matrix(as.numeric(stomach_logFC_mat$logFC),nrow=16794,ncol=1)
rownames(stomach_logFC) <- rownames(stomach_logFC_mat)

thyroid_logFC_mat       <- thyroid_logFC_mat[order(rownames(thyroid_logFC_mat)),]
thyroid_logFC           <- as.matrix(as.numeric(thyroid_logFC_mat$logFC),nrow=16794,ncol=1)
rownames(thyroid_logFC) <- rownames(thyroid_logFC_mat)


In [ ]:
logFC_mat = as.matrix(cbind(adipose_tissue_logFC,
             adrenal_gland_logFC,
             bladder_logFC,
             blood_logFC,
             blood_vessel_logFC,
             brain_logFC,
             breast_logFC,
             colon_logFC,
             esophagus_logFC,
             heart_logFC,
             kidney_logFC,
             liver_logFC,
             lung_logFC,
             muscle_logFC,
             nerve_logFC,
             pancreas_logFC,
             pituitary_logFC,
             salivary_gland_logFC,
             skin_logFC,
             small_intestine_logFC,
             spleen_logFC,
             stomach_logFC,
             thyroid_logFC),nrow=16794,ncol=23)


In [ ]:
rownames(logFC_mat) <- rownames(thyroid_logFC)
colnames(logFC_mat) <- c("adipose","adrenal_gland","bladder","blood","blood_vessel",
                        "brain","breast","colon","esophagus","heart","kidney",
                        "liver","lung","muscle","nerve","pancreas","pitutary",
                        "salivary_gland","skin","small_intestine",
                        "spleen","stomach","thyroid")
dim(logFC_mat)
head(logFC_mat)



In [ ]:
logFC_mat_NQ <- normalizeQuantiles(logFC_mat)

In [ ]:
dist_mat <- as.matrix(cor(logFC_mat_NQ))

In [ ]:
rownames(dist_mat) <- colnames(logFC_mat)
colnames(dist_mat) <- colnames(logFC_mat)

In [ ]:
library(pheatmap)
hm <- pheatmap(as.matrix(dist_mat))


In [ ]:
class(hm)